<a href="https://colab.research.google.com/github/Mardav15/oaqjp-final-project-emb-ai/blob/main/langchain_calc_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from langchain_core.tools import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

@tool
def multiply(first_int: int, second_int: int) -> int:
  """Multiply two integers together."""
  return first_int * second_int

@tool
def sqrt(first_int: int, second_int: int) -> int:
  """Find the root - x ^(1/y)."""
  return first_int ** (1/second_int)
@tool
def add(first_int: int, second_int: int) -> int:
  "Add two integers."
  return first_int + second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
  "Exponentiate the base to the exponent power."
  return base**exponent
tools = [multiply, sqrt, add, exponentiate]

In [9]:
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

import google.generativeai as genai
genai.configure(api_key=GOOGLE_API_KEY)

In [11]:
#!pip install -U langchain_google_genai
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)

llm_with_tools = llm.bind_tools([multiply,sqrt])
msg = llm_with_tools.invoke("find square root of 58.")
msg.tool_calls
from operator import itemgetter

chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | sqrt
chain.invoke("find square root of 58.")

7.615773105863909

In [12]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [13]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "Take 3 and multiply with 24, then find square root of the result."
    }
)



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'second_int': 24, 'first_int': 3}`


72
Invoking: `sqrt` with `{'second_int': 2, 'first_int': 72}`


8.48528137423857The square root of 72 is approximately 8.49.


> Finished chain.


{'input': 'Take 3 and multiply with 24, then find square root of the result.',
 'output': 'The square root of 72 is approximately 8.49.\n'}

In [15]:
agent_executor.invoke(
{
    "input": "2 ** 5 + 4 *6 and then squared root of the result =?. Think step by step."
 }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'exponent': 5, 'base': 2}`


32
Invoking: `multiply` with `{'second_int': 6, 'first_int': 4}`


24
Invoking: `add` with `{'second_int': 24, 'first_int': 32}`


56
Invoking: `sqrt` with `{'second_int': 2, 'first_int': 56}`


7.483314773547883The answer is approximately 7.48.  Here's how we got there:

1. **2<sup>5</sup>:** 2 raised to the power of 5 is 32.
2. **4 * 6:** 4 multiplied by 6 is 24.
3. **32 + 24:** 32 plus 24 is 56.
4. **√56:** The square root of 56 is approximately 7.48.


> Finished chain.


{'input': '2 ** 5 + 4 *6 and then squared root of the result =?. Think step by step.',
 'output': "The answer is approximately 7.48.  Here's how we got there:\n\n1. **2<sup>5</sup>:** 2 raised to the power of 5 is 32.\n2. **4 * 6:** 4 multiplied by 6 is 24.\n3. **32 + 24:** 32 plus 24 is 56.\n4. **√56:** The square root of 56 is approximately 7.48.\n"}